In [2]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 11.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 req

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import gensim.downloader as api
import contractions

In [5]:
import gensim.downloader as api

# Load the pre-trained Google News Word2Vec model
model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [17]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [7]:
# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

In [8]:
#pre-processing the tweet
def preprocess_tweet(tweet):
  tweet=tweet.lower() # converting to lowercase
  tweet = re.sub(r'http\S+|www\S+', '', tweet) #removing urls
  tweet = re.sub(r'@\w+|#\w+', '', tweet) #removing usernames and hashtags
  tweet=contractions.fix(tweet) #expand contrations
  tweet = re.sub(r'[^\w\s]', '', tweet) #remove punctuation
  tokens=word_tokenize(tweet)
  tokens=[lemmatizer.lemmatize(word) for word in tokens]
  stop_words = set(stopwords.words('english'))
  tokens = [word for word in tokens if word not in stop_words]

  return tokens

In [14]:
#converting tweets to word vectors
def tweet_to_wordvec(tweet, model):
  tokens=preprocess_tweet(tweet)
  vectors=[]
  for word in tokens:
    if word in model:
      vectors.append(model[word])
  if len(vectors)==0:
    return np.zeros(model.vector_size)

  return np.mean(vectors, axis=0)

In [11]:
df = pd.read_csv("Tweets.csv")

In [12]:
df = df[['airline_sentiment', 'text']]  # Adjust column names as necessary
df.columns = ['Sentiment', 'Tweet']

In [18]:
X = np.array([tweet_to_wordvec(tweet, model) for tweet in df['Tweet']])
y = df['Sentiment'].map({'positive': 0, 'neutral': 1, 'negative': 2})

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [21]:
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 77.56%


In [22]:
def predict_tweet_sentiment(model, w2v_model, tweet):
    # Convert the tweet to a vector
    tweet_vec = tweet_to_wordvec(tweet, w2v_model)
    # Predict using the trained model
    prediction = model.predict([tweet_vec])
    # Map the prediction back to sentiment labels
    sentiment_map = {0: 'positive', 1: 'neutral', 2: 'negative'}
    return sentiment_map[prediction[0]]

In [27]:
test_tweet = "The food was #bad but overall a good flight #safe @Delta"
predicted_sentiment = predict_tweet_sentiment(classifier, model, test_tweet)
print(f"Predicted sentiment: {predicted_sentiment}")

Predicted sentiment: positive
